ref:
1. [LangChain 怎麼玩？ 動態修改運作中的 Chain 設定 / configure chain internals at runtime](https://myapollo.com.tw/blog/langchain-configure-chain-at-runtime/)

2. [LangChain doc](https://python.langchain.com/v0.1/docs/expression_language/primitives/configure/)

在LangChain中可以用`configurable_fields`以及`configurable_alternatives`，動態修改chain的設定，或者置換chain上的某個Runnable(prompt, model, parser...)


In [1]:
from langchain_community.llms.ollama import Ollama
llm = Ollama(model='llama3.1', temperature=0)

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("user", "{input}"),
])
chain = prompt | llm
print(chain.invoke({'input': 'Tell me a joke'}))

Here's one:

What do you call a fake noodle?

(wait for it...)

An impasta!

Hope that made you laugh! Do you want to hear another one?


"如果我們想動態設定一些隨機性的話，雖然可以每次產生回應都重新載入語言模型並重新設定 temperature 參數，但是也會導致回應時間拉長，因為語言模型的載入會需要一點點時間，也許比較好的方式是讓 chain 具有動態修改設定的能力；又或者我們想做些實驗比較不同 prompt / 參數 / 模型的差異，這時候也需要動態修改設定的能力，讓我們在呼叫 chain 時使用不同的參數，以對結果進行比較；甚至是讓 chain 具有服務多名使用者的能力，針對使用者帳號不同載入不同的語言模型、對話紀錄，就像 ChatGPT 一樣，所有使用者之間不會看到彼此的對話紀錄以確保隱私，而且付費使用者多了可以切換至更強大的語言模型的功能。

針對這些需求， LangChain 為每個 Runnable 提供 1 個稱為 configurable_fields 的方法，讓我們可以動態修改 chain 上的設定值。"

### Example 1:

In [10]:
from langchain_core.runnables import ConfigurableField
from langchain_community.llms.ollama import Ollama

model = Ollama(model="lamma3.1", temperature=0).configurable_fields(
    model=ConfigurableField(
        id="model",
        name="Model",
        description="The model to use for the chat",
    )
)

print(model.model)

model_m = model.with_config(configurable={"model": "mistral"})

# QUESTION: why
print(model_m.model)

lamma3.1
lamma3.1


### Example 2:

In [11]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import ConfigurableField

# from langchain_community.llms.ollama import Ollama
# llm = Ollama(model='llama3.1', temperature=0).configurable_fields(
#     temperature=ConfigurableField(
#         id="llm_temperature",
#         name="LLM temperature",
#         description="The temperature of the LLM",
#     )
# )

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model='gpt-4o', temperature=0).configurable_fields(
    temperature=ConfigurableField(
        id="llm_temperature",
        name="LLM temperature",
        description="The temperature of the LLM",
    )
)

prompt = ChatPromptTemplate.from_messages([
    ("user", "{input}"),
])

chain = prompt | llm

chain_t09 = chain.with_config(configurable = {'llm_temperature': 1.0})

res = chain_t09.invoke({'input': 'Tell me a joke'})

print(res)

content="Sure, here are three jokes for you:\n\n1. **Why don't scientists trust atoms?**\n   Because they make up everything!\n\n2. **Why did the scarecrow win an award?**\n   Because he was outstanding in his field!\n\n3. **Why don't skeletons fight each other?**\n   They don't have the guts!" additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 67, 'prompt_tokens': 12, 'total_tokens': 79}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_157b3831f5', 'finish_reason': 'stop', 'logprobs': None} id='run-9d0c0eeb-709c-4dc9-bf21-cbbd22e2561c-0' usage_metadata={'input_tokens': 12, 'output_tokens': 67, 'total_tokens': 79}


### Exampl 3:

In [23]:
from langchain_community.llms.ollama import Ollama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import ConfigurableField

llm = Ollama(model='llama3.1', temperature=0).configurable_fields(
    temperature=ConfigurableField(
        id="llm_temperature",
        name="LLM temperature",
        description="The temperature of the LLM",
    ),
    model=ConfigurableField(
        id="llm_model",
        name="LLM model",
        description="The model of the LLM",
    )
)

prompt  = ChatPromptTemplate.from_messages([
    ('user', '{input}'),
])

chain = prompt | llm

chain_t09 = chain.with_config(configurable = {'llm_temperature': 1.0, 'llm_model': 'mistral'})

res = chain_t09.invoke({'input': 'Tell me a joke'})

print(res)

llama3.1
 Of course! Here's one for you: Why don't scientists trust atoms?

Because they make up everything... except secret laboratory recipes!


## 用 configurable_alternatives 動態置換 Runnable 
先前的範例都是使用 LLaMA 系列的語言模型，這些系列的語言模型都可以使用 Ollama 載入，所以可以使用 configurable_fields() 動態設定語言模型，不過 OpenAI 的語言模型就不能使用 Ollama 載入，這使得我們無法使用 configurable_fields() 方法動態切換語言模型。

LangChain 還有 1 個稱為 configurable_alternatives() 的方法，可以動態置換 Runnable, 也就是說我們可以透過 configurable_alternatives() 方法，動態把 Ollama(model='llama2') 換成 ChatOpenAI(model="gpt-3.5-turbo", temperature=0), 其範例如下：

### Example 1

In [12]:
from langchain_community.llms.ollama import Ollama
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import ConfigurableField

llm = Ollama(model='llama3.1').configurable_alternatives(
    ConfigurableField(id="llm"),
    default_key='llama3.1',
    gpt4o=ChatOpenAI(model="gpt-4o", temperature=0),
)

prompt = ChatPromptTemplate.from_messages([
    ("user", "{input}"),
])
chain = prompt | llm

print(chain.with_config(configurable={"llm": "gpt4o"}).invoke({'input': 'Tell me a joke'}))
print(chain.invoke({'input': 'Tell me a joke'}))

content="Sure, here's a light-hearted joke for you:\n\nWhy don't skeletons fight each other?\n\nThey don't have the guts!" additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 11, 'total_tokens': 35}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_25624ae3a5', 'finish_reason': 'stop', 'logprobs': None} id='run-23d4ce30-595f-4e4c-9282-40b545ceb6a6-0' usage_metadata={'input_tokens': 11, 'output_tokens': 24, 'total_tokens': 35}
Here's one:

What do you call a fake noodle?

(wait for it...)

An impasta!

Hope that made you laugh! Do you want to hear another?


### Example 2

In [19]:
# from langchain_community.llms.ollama import Ollama
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import ConfigurableField

llm = ChatOpenAI(model='gpt-4o-mini').configurable_alternatives(
    ConfigurableField(id="llm"),
    default_key='gpt-4o-mini',
    gpt4o=ChatOpenAI(model="gpt-4o", temperature=0),
)

prompt = ChatPromptTemplate.from_messages([
    ("user", "{input}"),
])
chain = prompt | llm

print(chain.with_config(configurable={"llm": "gpt4o"}).invoke({'input': 'Tell me a joke'}))
print(chain.invoke({'input': 'Tell me a joke'}))

content="Sure, here's one for you:\n\nWhy don't skeletons fight each other?\n\nThey don't have the guts!" additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 11, 'total_tokens': 32}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_25624ae3a5', 'finish_reason': 'stop', 'logprobs': None} id='run-7b3a5150-c7fd-497a-abe2-9e0cb9792d45-0' usage_metadata={'input_tokens': 11, 'output_tokens': 21, 'total_tokens': 32}
content='Why did the scarecrow win an award? \n\nBecause he was outstanding in his field!' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 11, 'total_tokens': 29}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_483d39d857', 'finish_reason': 'stop', 'logprobs': None} id='run-741a7b28-c0f0-42fc-a949-22c7602e757c-0' usage_metadata={'input_tokens': 11, 'output_tokens': 18, 'total_tokens': 29}


上面的examples中，利用`configurable_alternatives`方法設定`ConfigurableField`。

然候再用`.with_config`切換`Runnable`。

上面的examples是切換模型，而我們也可以切換prompt

In [24]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o", temperature=0)

prompt = PromptTemplate.from_template(
    "temll me a joke about {topic}",
).configurable_alternatives(
    ConfigurableField(id="prompt"),
    default_key="joke",
    poem=PromptTemplate.from_template("Write a poem about {topic}"),
    lyrics=PromptTemplate.from_template("Write a song about {topic}"),
)

chain = prompt | llm

res = chain.invoke({"topic": "Earth"})
print(res, "\n============================\n")

res = chain.with_config(configurable={"prompt": "poem"}).invoke({"topic": "Earth"})
print(res, "\n============================\n")

res = chain.with_config(configurable={"prompt": "lyrics"}).invoke({"topic": "Earth"})
print(res)

content="Sure, here's a joke about Earth for you:\n\nWhy did the Earth break up with the other planets?\n\nBecause they all had too much space!" additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 14, 'total_tokens': 43}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_25624ae3a5', 'finish_reason': 'stop', 'logprobs': None} id='run-fd292406-117a-4007-8c08-f8012aeab6f5-0' usage_metadata={'input_tokens': 14, 'output_tokens': 29, 'total_tokens': 43} 

content="In the cradle of the cosmos, blue and green,\nA jewel spins, in stardust sheen,\nEarth, our home, where life does teem,\nA living, breathing, ancient dream.\n\nMountains rise with majesty,\nValleys whisper history,\nOceans dance in rhythmic grace,\nReflecting skies, a vast embrace.\n\nForests hum with secrets old,\nLeaves of emerald, stories told,\nRivers carve their timeless path,\nIn nature's art, no aftermath.\n\nDeserts stretch in golden hues,\nUnderneath 

## 動態載入不同使用者的對話紀錄 
我們在使用 ChatGPT 時， ChatGPT 會幫我們保留不同的對話紀錄，而且不會看到不屬於我們的對話紀錄，這也是做任何應用重要的功能—針對不同使用者載入專屬的設定、資料。

學會使用 configurable_fields() 與 configurable_alternatives() 之後，我們就可以動態載入使用者的相關設定與資料。

舉對話紀錄(chat history)為例， LangChain 稱此功能為 memory, 我們可以將對話紀錄存在各種 memory 中，例如檔案、資料庫等等， LangChain 提供多種整合方案可以選擇，詳細可以閱讀此文件。

本文僅展示以 Python dictionary 儲存對話紀錄作為教學範例。

LangChain 其實有提供 1 個 Runnable 稱為 RunnableWithMessageHistory, 這個類別其實就是使用 configurable_fields() 方法，預設讓我們可以動態設定 session_id 藉此載入不同對話紀錄。

以下是使用 RunnableWithMessageHistory 的範例程式碼：

In [25]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.llms.ollama import Ollama


chat001 = ChatMessageHistory()
chat001.add_user_message('My name is Amo.')

store = {
    'chat001': chat001,
}


def get_chat_history(session_id: str) -> BaseChatMessageHistory:
    # Create a new chat history if it doesn't exist
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


llm = Ollama(model='llama3.1')

prompt = ChatPromptTemplate.from_messages([
    ('system', 'You are a good assistant.'),
    MessagesPlaceholder(variable_name='chat_history'),
    ('user', '{input}'),
])

chain = prompt | llm

with_message_history = RunnableWithMessageHistory(
    chain,
    get_chat_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

session_id = 'chat001'
input_text = input('>>> ')
while input_text.lower() != 'bye':
    if input_text:
        response = with_message_history.with_config(
            configurable={'session_id': session_id}
        ).invoke({'input': input_text})
        print(response)
    input_text = input('>>> ')